<a href="https://colab.research.google.com/github/JuanCastrillonL/Data-Integration-Analytics/blob/main/Reto_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
path = "/content/sample_data/4. DB_Fintechs (USD).xlsx"

df = pd.read_excel(path,sheet_name=0)
df.head()

list(df.columns)

XDB = df[['Edad',  'Ingresos',  'Egresos', 'Monto (EAD)']]#variables de interes
XDB.dropna()
XDB.head()

,Edad,Ingresos,Egresos,Monto (EAD)
0,33,496.87275,550.117556,250.209001
1,61,518.90575,240.179333,574.469987
2,29,1305.97300,769.749778,1469.739871
3,39,194.06275,188.418222,76.842043
4,38,883.16350,1059.917333,440.780558


In [ ]:
XDBM  = np.max(XDB, axis=0) #maximos
XDBM

XDBm = np.min(XDB, axis=0) #minimos
XDBm

np.random.seed(42)

#semillas (serian 5 en este caso) ya que el ejercicio nos pide crear 5 clusters
XC = np.zeros((5,4))

for i in range (5): #numero de montones de datos
  for j in range (4): #cada monton va a tener 5 variables
    XC[i,j] = np.random.uniform(XDBm[j],XDBM[j])

XDB = np.array(XDB)
XCd = np.zeros((5,4))
ND = np.zeros((5,1))


#clasificacion de los datos por monton

for k in range(len(XDB)):
  d = np.sum(np.abs(XC[:,:]-XDB[k,:]), axis=1) # distancia de un registro a un cluster
  fila = np.argmin(d) # numero de cluster al que pertenece un dato
  XCd[fila,]= XCd[fila,]+XDB[k,:] #sumo la info de las variables a los clusters
  ND[fila,]=ND[fila,]+1 #cuanto el numero de datos que quedaron por cada cluster

print("el numero de datos por cluster es:\n", ND )



el numero de datos por cluster es:
 [[  38.]
 [   0.]
 [   7.]
 [1112.]
 [ 343.]]


Como se esta generando un cluster vacio, necesito asignarle a ese cluster un valor de centro y esta asignacion se hace con un punto de XDB, para poder realizar el proceso de integracion y el cluster ahora tiene al menos un dato

In [ ]:
clusters_vacios = np.where(ND == 0)[0]  # Índices de clusters vacíos

if len(clusters_vacios) > 0:
    print(f"Se encontraron {len(clusters_vacios)} clusters vacíos, reasignando centros...")
    for i in clusters_vacios:
        nuevo_centro = XDB[np.random.randint(0, len(XDB))]  # Escoge un punto aleatorio como nuevo centro
        XC[i, :] = nuevo_centro
        print(f"Nuevo centro asignado al cluster {i}: {nuevo_centro}")



ND[ND == 0] = 1  # Evita que haya divisiones por cero artificialmente
XCd = XCd / ND

dfxco= pd.DataFrame((XCd.astype(float)))
dfxco.columns=['Edad', 'Ingresos', 'Egresos', 'Monto (EAD)']
dfxco.head(5)

print("Número de datos por cluster inicial:", ND.astype(int).flatten())


Número de datos por cluster inicial: [  38    1    7 1112  343]


Integracion de datos de la primera fintch externa

In [ ]:
dfe1 = pd.read_excel(path,sheet_name=1) #datos correspondientes a la primera fintech externa
dfe1.head()

XDBe1 = np.array(dfe1[['Edad',  'Ingresos',  'Egresos', 'Monto (EAD)']].dropna())#variables de interes

Xco1 = np.copy(XCd) #clusters estaticos/iniciales
NDe1 = np.zeros((5,1)) #numero de datos por cluster

for k in range(len(XDBe1)):
  de = np.sqrt(np.sum((Xco1[:,:]-XDBe1[k,:])**2, axis=1)) #voy sumando la informacion de las varuables a los clusters
  fila = np.argmin(de)
  Xco1[fila,]= (Xco1[fila,]+XDBe1[k,:])/2 #sumo la info de las variables a los clusters

  NDe1[fila,]=NDe1[fila,]+1

print("el numero de datos integrados por cluster desde la primera fintech externa es:\n", NDe1 )

el numero de datos integrados por cluster desde la primera fintech externa es:
 [[ 93.]
 [479.]
 [163.]
 [503.]
 [263.]]


Integracion de los datos desde la segunda fintech externa

In [ ]:
dfe2 = pd.read_excel(path,sheet_name=2) #datos correspondientes a la segunda fintech externa
dfe2.head()

XDBe2 = np.array(dfe2[['Edad',  'Ingresos',  'Egresos', 'Monto (EAD)']].dropna())#variables de interes

Xco2 = np.copy(XCd) #clusters estaticos/iniciales
NDe2 = np.zeros((5,1)) #numero de datos por cluster

for k in range(len(XDBe2)):
  de = np.sqrt(np.sum((Xco2[:,:]-XDBe2[k,:])**2, axis=1)) #voy sumando la informacion de las varuables a los clusters
  fila = np.argmin(de)
  Xco2[fila,]= (Xco1[fila,]+XDBe2[k,:])/2 #sumo la info de las variables a los clusters

  NDe2[fila,]=NDe2[fila,]+1


print("el numero de datos integrados por cluster desde la primera fintech externa es:\n", NDe2 )

el numero de datos integrados por cluster desde la primera fintech externa es:
 [[ 92.]
 [144.]
 [ 17.]
 [726.]
 [521.]]


Integracion de datos desde la tercera fintech observada

In [ ]:
dfe3 = pd.read_excel(path, sheet_name=3)  # Cambiado a dfe3 y sheet_name=2
dfe3.head()

XDBe3 = np.array(dfe3[['Edad', 'Ingresos', 'Egresos', 'Monto (EAD)']].dropna())  # Cambiado a XDBe3
Xco3 = np.copy(XCd)  # Clusters estáticos/iniciales, cambiado a Xco3
NDe3 = np.zeros((5, 1))  # Número de datos por cluster, cambiado a NDe3

for k in range(len(XDBe3)):
    de = np.sqrt(np.sum((Xco3[:, :] - XDBe3[k, :]) ** 2, axis=1))
    fila = np.argmin(de)
    Xco3[fila, :] = (Xco3[fila, :] + XDBe3[k, :]) / 2  # Actualizar centros de cluster

    NDe3[fila, 0] = NDe3[fila, 0] + 1

print("El número de datos integrados por cluster desde la segunda fintech externa es:\n", NDe3)

El número de datos integrados por cluster desde la segunda fintech externa es:
 [[115.]
 [500.]
 [268.]
 [281.]
 [178.]]


In [ ]:
# Total de puntos de datos después de cada integración
total_datos_despues_integracion1 = np.sum(ND) + np.sum(NDe1)
print(f"Total de puntos de datos después de la primera integración: {total_datos_despues_integracion1}")
print(f"Puntos de datos por clúster después de la primera integración:\n{ND + NDe1}")

total_datos_despues_integracion2 = total_datos_despues_integracion1 + np.sum(NDe2)
print(f"Total de puntos de datos después de la segunda integración: {total_datos_despues_integracion2}")
print(f"Puntos de datos por clúster después de la segunda integración:\n{ND + NDe1 + NDe2}")

total_datos_despues_integracion3 = total_datos_despues_integracion2 + np.sum(NDe3)
print(f"Total de puntos de datos después de la tercera integración: {total_datos_despues_integracion3}")
print(f"Puntos de datos por clúster después de la tercera integración:\n{ND + NDe1 + NDe2 + NDe3}")




Total de puntos de datos después de la primera integración: 3002.0
Puntos de datos por clúster después de la primera integración:
[[ 131.]
 [ 480.]
 [ 170.]
 [1615.]
 [ 606.]]
Total de puntos de datos después de la segunda integración: 4502.0
Puntos de datos por clúster después de la segunda integración:
[[ 223.]
 [ 624.]
 [ 187.]
 [2341.]
 [1127.]]
Total de puntos de datos después de la tercera integración: 5844.0
Puntos de datos por clúster después de la tercera integración:
[[ 338.]
 [1124.]
 [ 455.]
 [2622.]
 [1305.]]
